In [2]:
import numpy as np

print('USN: 1BM23CS245, NAME: PRATHEEKSHA PAI')

# Packet delays in ms (transmission times)
packet_delays = [10, 20, 5, 15, 25]

# Base drop probability for packets (could be different per packet)
base_drop_probs = [0.05, 0.10, 0.02, 0.07, 0.12]

# Congestion factor (increases drop probability based on packet position)
congestion_factor = 0.01

# Objective function: combine total expected drops and total completion time
def objective(order, alpha=1000):
    total_time = 0
    current_time = 0
    total_drops = 0

    for position, pkt_idx in enumerate(order):
        current_time += packet_delays[pkt_idx]
        total_time += current_time

        # Calculate drop probability increasing with position in queue
        drop_prob = base_drop_probs[pkt_idx] + congestion_factor * position
        total_drops += drop_prob

    return alpha * total_drops + total_time

# Generate a random solution (permutation of packets)
def random_solution(n):
    solution = np.arange(n)
    np.random.shuffle(solution)
    return solution

# Levy flight step (swap two packets)
def levy_flight(solution):
    n = len(solution)
    i, j = np.random.choice(n, 2, replace=False)
    new_solution = solution.copy()
    new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
    return new_solution

def cuckoo_search(n_packets, n_nests=15, n_iterations=100, pa=0.25):
    nests = [random_solution(n_packets) for _ in range(n_nests)]
    fitness = np.array([objective(nest) for nest in nests])

    best_idx = np.argmin(fitness)
    best_nest = nests[best_idx].copy()
    best_fitness = fitness[best_idx]

    for iter in range(n_iterations):
        for i in range(n_nests):
            new_nest = levy_flight(nests[i])
            new_fitness = objective(new_nest)

            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness

                if new_fitness < best_fitness:
                    best_fitness = new_fitness
                    best_nest = new_nest.copy()

        for i in range(n_nests):
            if np.random.rand() < pa:
                nests[i] = random_solution(n_packets)
                fitness[i] = objective(nests[i])

                if fitness[i] < best_fitness:
                    best_fitness = fitness[i]
                    best_nest = nests[i].copy()

        if iter % 10 == 0 or iter == n_iterations - 1:
            print(f"Iteration {iter+1}/{n_iterations}, Best fitness: {best_fitness:.4f}")

    return best_nest, best_fitness

def print_results(order):
    total_time = 0
    current_time = 0
    drop_probs = []

    print("\nPacket order (indices):", order)
    print("Packet | Position | Delay (ms) | Drop Probability")
    print("-----------------------------------------------")

    for position, pkt_idx in enumerate(order):
        current_time += packet_delays[pkt_idx]
        drop_prob = base_drop_probs[pkt_idx] + congestion_factor * position
        drop_probs.append(drop_prob)
        print(f"{pkt_idx:^6} | {position:^8} | {packet_delays[pkt_idx]:^10} | {drop_prob:.4f}")

    total_drops = sum(drop_probs)
    total_time = sum(np.cumsum([packet_delays[i] for i in order]))

    # Calculate throughput: expected successful packets / total_time
    expected_success_packets = sum(1 - dp for dp in drop_probs)
    throughput = expected_success_packets / total_time if total_time > 0 else 0

    print("-----------------------------------------------")
    print(f"Total expected packet drops: {total_drops:.4f}")
    print(f"Total transmission time (sum of completion times): {total_time} ms")
    print(f"Expected successful packets: {expected_success_packets:.4f}")
    print(f"Throughput (successful packets per ms): {throughput:.6f}")

# Run optimization
best_order, best_cost = cuckoo_search(len(packet_delays))

print("\nOptimization complete.")
print(f"Objective function value (lower is better): {best_cost:.4f}")
print_results(best_order)


USN: 1BM23CS245, NAME: PRATHEEKSHA PAI
Iteration 1/100, Best fitness: 640.0000
Iteration 11/100, Best fitness: 635.0000
Iteration 21/100, Best fitness: 635.0000
Iteration 31/100, Best fitness: 635.0000
Iteration 41/100, Best fitness: 635.0000
Iteration 51/100, Best fitness: 635.0000
Iteration 61/100, Best fitness: 635.0000
Iteration 71/100, Best fitness: 635.0000
Iteration 81/100, Best fitness: 635.0000
Iteration 91/100, Best fitness: 635.0000
Iteration 100/100, Best fitness: 635.0000

Optimization complete.
Objective function value (lower is better): 635.0000

Packet order (indices): [2 0 3 1 4]
Packet | Position | Delay (ms) | Drop Probability
-----------------------------------------------
  2    |    0     |     5      | 0.0200
  0    |    1     |     10     | 0.0600
  3    |    2     |     15     | 0.0900
  1    |    3     |     20     | 0.1300
  4    |    4     |     25     | 0.1600
-----------------------------------------------
Total expected packet drops: 0.4600
Total transmis